In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

# Qiskit Imports (Quantum Optimization)
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import Aer
from qiskit_aer.primitives import Sampler  

# Step 1: Download Historical Stock Data
stocks = ["TCS.NS", "INFY.NS", "HDFCBANK.NS", "RELIANCE.NS", "ITC.NS", "BAJFINANCE.NS"]
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")['Close']

# Step 2: Calculate Returns & Covariance Matrix
returns = df.pct_change().dropna()
expected_returns = returns.mean()
cov_matrix = returns.cov()

# Step 3: Construct QUBO Matrix
assets = len(expected_returns)
Q = np.zeros((assets, assets))
penalty = 1 / np.mean(abs(cov_matrix.values))  # Adjust for risk-return balance

for i in range(assets):
    Q[i, i] = -expected_returns.iloc[i] + penalty * cov_matrix.iloc[i, i]
    for j in range(i + 1, assets):
        Q[i, j] = penalty * cov_matrix.iloc[i, j]
        Q[j, i] = Q[i, j]

# Step 4: Classical Optimization (Scipy)
def classical_solver(Q):
    def objective(x):
        return sum(Q[i, j] * x[i] * x[j] for i in range(assets) for j in range(assets))
    
    x0 = np.random.randint(0, 2, assets)  # Initial random binary solution
    bounds = [(0, 1) for _ in range(assets)]  # Binary constraint
    result = minimize(objective, x0, bounds=bounds, method='COBYLA')
    
    # Convert floating values to binary (0 or 1)
    x_opt = np.round(result.x).astype(int)
    return x_opt, result.fun

classical_solution, classical_value = classical_solver(Q)

# Step 5: Quantum Optimization (QAOA on IBMQ)
# Define QUBO problem for Qiskit
qubo = QuadraticProgram()
for i in range(assets):
    qubo.binary_var(f"x{i}")

linear_terms = {f"x{i}": Q[i, i] for i in range(assets)}
quadratic_terms = {(f"x{i}", f"x{j}"): Q[i, j] for i in range(assets) for j in range(i + 1, assets)}
qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)

# Load IBMQ backend
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Session, Options, Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import QAOAAnsatz


####### Real Backend

# QiskitRuntimeService.save_account(
#             channel="ibm_quantum",
#             token="ef7cd82468de50529639b7e506dab7dbb30cc6ed636948a42181ca9ef65744e57e24c53f0d366eef7aec01a9f34819e79b777a7e6beb0fc618f9f5a92976922d",
#             overwrite=True)

# service = QiskitRuntimeService()
# backend = service.backend("ibm_sherbrooke")

# qubit_op = SparsePauliOp.from_list([("ZZII", 1), ("IZZI", 1), ("IIZZ", 1), ("ZIIZ", 1)])

# ansatz = QAOAAnsatz(qubit_op, reps=10)
# pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
# transpiled_ansatz = pm.run(ansatz)

# options = Options(
#     # Execution settings
#     default_shots=1000,  # Direct shot specification
    
#     # Resilience settings (error mitigation)
#     resilience= {
#         "level": 1  # Nested under 'resilience' key
#     }
# )

# with Session(backend=backend) as session:  
#     sampler = SamplerV2(options=options)

    # qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=10)
    # meo = MinimumEigenOptimizer(qaoa)
    # qaoa_result = meo.solve(qubo)

    # qaoa_solution = np.array([int(qaoa_result.x[i]) for i in range(assets)])
    # qaoa_value = qaoa_result.fval


# backend = provider.get_backend("ibmq_qasm_simulator")  # Change to real backend like 'ibmq_mumbai' if available
# sampler = Aer.get_backend('aer_simulator')  # Local simulator for testing


#######  Fake Backend

from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2  
from qiskit_ibm_runtime.options import SamplerOptions

fake_backend = FakeMumbaiV2()

pm = generate_preset_pass_manager(
    backend=fake_backend,
    optimization_level=1,  
    basis_gates=["rz", "sx", "cx", "x"]  
)

num_qubits = 27  # Match your problem size
mixer_operator = SparsePauliOp("X" * num_qubits)

qubit_op = SparsePauliOp.from_list([
    ("Z" + "I" * 26, 1),               # Z on qubit 0
    ("ZZ" + "I" * 25, 1),              # ZZ on qubits 0,1
    ("IZZI" + "I" * 23, 1),            # ZZ on qubits 1,2
    ("IIZZ" + "I" * 23, 1),            # ZZ on qubits 2,3
    ("ZIIZ" + "I" * 23, 1)             # Z on qubit 0 and 3
])
ansatz = QAOAAnsatz(qubit_op, reps=10,mixer_operator=mixer_operator)

# print(f"Original parameters: {ansatz.num_parameters}")  
transpiled_ansatz = pm.run(ansatz)
# print(f"Transpiled parameters: {transpiled_ansatz.num_parameters}") 

# from qiskit import transpile 
# transpiled_ansatz = transpile(
#     ansatz,
#     backend=fake_backend,
#     basis_gates=["rz", "sx", "cx", "x"],
#     optimization_level=1,
#     layout_method="trivial",  # Avoid qubit remapping issues
# )

sampler = Sampler(
    mode=fake_backend,
    options=SamplerOptions(
            default_shots=1024,
            execution={"init_qubits": True}
    )
)

# from sampleradapter import SamplerV2Adapter
# sampler = SamplerV2Adapter(sampler)

qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=10)
qaoa.ansatz = transpiled_ansatz
qaoa.initial_point = [0.1] * transpiled_ansatz.num_parameters 

# print(transpiled_ansatz.draw())

meo = MinimumEigenOptimizer(qaoa)
qaoa_result = meo.solve(qubo)


qaoa_solution = np.array([int(qaoa_result.x[i]) for i in range(assets)])
qaoa_value = qaoa_result.fval





# Step 6: Print Results
print("\n===== Portfolio Optimization Results =====")
print("Classical Optimization:")
print("Selected Stocks:", [stocks[i] for i in range(assets) if classical_solution[i] == 1])
print("Objective Value:", classical_value)

print("\nQAOA Quantum Optimization (IBMQ):")
print("Selected Stocks:", [stocks[i] for i in range(assets) if qaoa_solution[i] == 1])
print("Objective Value:", qaoa_value)

# Step 7: Visualization
plt.figure(figsize=(10, 5))
methods = ["Classical", "QAOA (IBMQ)"]
values = [classical_value, qaoa_value]

sns.barplot(x=methods, y=values, palette=["blue", "green"])
plt.title("Portfolio Optimization: Classical vs Quantum")
plt.ylabel("Objective Function Value (Lower is Better)")
plt.show()

[*********************100%***********************]  6 of 6 completed


TypeError: SamplerV2.run() takes 2 positional arguments but 3 were given

In [54]:
from qiskit import *
from qiskit.quantum_info import SparsePauliOp

qubits = QuantumRegister(127)
circuit = QuantumCircuit(qubits)

circuit.h(qubits[0])
circuit.cx(qubits[0],qubits[1])

observables = SparsePauliOp.from_list([
    ("Z" + "I"*126, 1.0),
    ("I" + "Z" + "I"*125, 1.0),
    ("Z" + "Z" + "I"*125, 1.0),
])


from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeSherbrooke()

pass_manager = generate_preset_pass_manager(
    optimization_level=3,
    backend=backend,
)

isa_circuit = pass_manager.run(circuit)

isa_circuit.draw("mpl",idle_wires=False, scale=0.5)

from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)

pub = (isa_circuit,observables)
job = estimator.run([pub])
result = job.result()
pub_result = result[0]
expectation_val = float(pub_result.data.evs)


In [55]:
expectation_val

2.8505859375